# 401 Assignment 2

In [47]:
# imports
import random
import math
import numpy as np
import torch
import torchvision
import torchvision.transforms
import torch.optim
import torch.utils.data.sampler
import torch.nn.functional
from torch.autograd import Variable
import time

# settings

random.seed(55)

## 1. Gradient-based Learning with Tensors

### Defining R<sup>m</sup> to R<sup>n</sup> Function
 R<sup>m</sup> = 5 R<sup>n</sup> = 4 <br>
 

In [41]:
def create_output_tensor_noisy(input, weights):
    tensor_a = input#torch.from_numpy(input)
    tensor_b = torch.from_numpy(weights)
    output = tensor_a.t()@tensor_b.t()
    noise = np.random.normal(loc=0, scale=0.01, size=np.shape(input.size))
    output = output + torch.from_numpy(noise)
    return output

def mse(in1, in2):
    diff = in1 - in2
    return torch.sum(diff*diff) / diff.numel()

def model(x,w,b):
    return x.t() @ w.t() +b

def train(input, target, w, b, i, learning_rate=1e-2):
    prediction = model(input, w,b)
    loss = mse(prediction, target)
    if i % 100 == 0:
        print("epoch " + str(i) + " loss = " + str(loss))
    loss.backward(retain_graph=True)
    with torch.no_grad():
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()

def test(input, target, w, b):
    prediction = model(input, w, b)
    print('Expected:')
    print(target.data.numpy())
    print('Prediction:')
    print(prediction.data.numpy())



eqtn = np.array([[3.7, 2.3, 1, 0, 5],
      [4.7, 8.1, 2.5,  0, 25],
      [2.7, 1, 9, 3.9, 0],
      [5.2, 6, 2, 0, 0]], dtype=np.float32)
tensor_in = torch.randn(5,100, requires_grad=True)
tensor_target = create_output_tensor_noisy(tensor_in, eqtn)     
weights = torch.randn(4,5, requires_grad=True)
biases = torch.randn(4, requires_grad=True)

for i in range(1,2001):
    train(input= tensor_in, target= tensor_target, w = weights, b = biases, i = i)

tensor_test_in = torch.randn(5,10, requires_grad=True)
tensor_test_target = create_output_tensor_noisy(tensor_test_in, eqtn)  
test(tensor_test_in, tensor_test_target, weights, biases)

print('**********')
print('known linear weights:')
print(eqtn)
print('****')
print('found linear weights:')
print(weights)
print('**********')


epoch 100 loss = tensor(88.3999, grad_fn=<DivBackward0>)
epoch 200 loss = tensor(32.7373, grad_fn=<DivBackward0>)
epoch 300 loss = tensor(12.3499, grad_fn=<DivBackward0>)
epoch 400 loss = tensor(4.7476, grad_fn=<DivBackward0>)
epoch 500 loss = tensor(1.8630, grad_fn=<DivBackward0>)
epoch 600 loss = tensor(0.7483, grad_fn=<DivBackward0>)
epoch 700 loss = tensor(0.3086, grad_fn=<DivBackward0>)
epoch 800 loss = tensor(0.1311, grad_fn=<DivBackward0>)
epoch 900 loss = tensor(0.0576, grad_fn=<DivBackward0>)
epoch 1000 loss = tensor(0.0261, grad_fn=<DivBackward0>)
epoch 1100 loss = tensor(0.0122, grad_fn=<DivBackward0>)
epoch 1200 loss = tensor(0.0059, grad_fn=<DivBackward0>)
epoch 1300 loss = tensor(0.0029, grad_fn=<DivBackward0>)
epoch 1400 loss = tensor(0.0015, grad_fn=<DivBackward0>)
epoch 1500 loss = tensor(0.0008, grad_fn=<DivBackward0>)
epoch 1600 loss = tensor(0.0004, grad_fn=<DivBackward0>)
epoch 1700 loss = tensor(0.0002, grad_fn=<DivBackward0>)
epoch 1800 loss = tensor(0.0001, grad

## 2. Transfer Learning


### MNIST

In [67]:
seed = 55
np.random.seed(seed)
torch.manual_seed(seed)

class MNISTCNN(torch.nn.Module):
    def __init__(self):
        super(MNISTCNN,self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.drop_out = torch.nn.Dropout()
        self.fc1 = torch.nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = torch.nn.Linear(1000, 10)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.drop_out(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

def train_net(mnist_model, device, train_loader, optimizer, loss_func, epoch):
    mnist_model.train()
    for i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = mnist_model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))

def test_net(mnist_model, device, test_loader, loss_func):
    mnist_model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = mnist_model(data)
            _, pred = torch.max(output.data,1)
            test_loss += loss_func(output, target).item() # sum up batch loss
            correct += pred.eq(target).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

batch_size = 64
train_batch_size = 64
test_batch_size = 100
epochs = 5
device = torch.device("cpu")
learning_rate = 1e-3
momentum = 0.5


train_loader = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST('../data', train=True, download=True,
               transform=torchvision.transforms.transforms.Compose([
                   torchvision.transforms.transforms.ToTensor(),
                   torchvision.transforms.transforms.Normalize((0.1307,), (0.3081,))
               ])),
            batch_size=train_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST('../data', train=False, transform=torchvision.transforms.transforms.Compose([
                torchvision.transforms.transforms.ToTensor(),
                torchvision.transforms.transforms.Normalize((0.1307,), (0.3081,))
            ])),
            batch_size=batch_size, shuffle=False)


mnist_model = MNISTCNN().to(device)
optimizer = torch.optim.Adam(mnist_model.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(1, epochs + 1):
    test_net(mnist_model,device,test_loader,loss_func)
    train_net(mnist_model, device, train_loader, optimizer, loss_func, epoch)
    test_net(mnist_model, device, test_loader, loss_func)


Test set: Average loss: 0.0364, Accuracy: 623/10000 (6%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320465
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.852432
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.443613
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.631681
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.279837
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.058500
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.214556
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.466350
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.201038
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.121833
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.091798
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.152624
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.190699
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.091981
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.310389
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.146955
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.182830
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.283139
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.279277
Train Epoch: 1 [1216

### CIFAR10

running cifar as a black and white image

In [77]:
seed = 55
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cpu")
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=28),
    torchvision.transforms.Grayscale(num_output_channels=1),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))])#((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Training
n_training_samples = 20000
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 5000
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 5000
test_sampler = torch.utils.data.sampler.SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))


def get_train_loader(batch):
    return torch.utils.data.DataLoader(train_set, batch_size=batch,sampler=train_sampler, num_workers=2)

cifar_test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
cifar_val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)
cifar_train_loader = get_train_loader(batch_size)


cifar_model = MNISTCNN().to(device)
cifar_optimizer = torch.optim.Adam(cifar_model.parameters(), lr=learning_rate)
cifar_loss_func = torch.nn.CrossEntropyLoss()


cifar_model_transfer = MNISTCNN().to(device)
cifar_model_transfer.layer1 = mnist_model.layer1
cifar_transfer_optimizer = torch.optim.Adam(cifar_model_transfer.parameters(), lr=learning_rate)
cifar_transfer_loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(1, epochs + 1):
    #test_net(cifar_model,device,test_loader,loss_func)
    train_net(cifar_model, device, cifar_train_loader, cifar_transfer_optimizer, loss_func, epoch)
    test_net(cifar_model, device, cifar_test_loader, loss_func)
    train_net(cifar_model_transfer, device, cifar_train_loader, cifar_transfer_optimizer, loss_func, epoch)
    test_net(cifar_model_transfer, device, cifar_test_loader, loss_func)
    


Files already downloaded and verified
Files already downloaded and verified


AssertionError: Torch not compiled with CUDA enabled